# Etapa de Tratamento dos Dados
Projeto Final - Resilia Educação & Ifood - #**VamoAI**


*Dashboards gerados por meio dos dados aqui tratados: < link do tableau>*


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)


Mounted at /content/drive
Mounted at /content/drive


## Bibliotecas

In [2]:
import pandas as pd
import numpy as np
import json
from pprint import pprint
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from PIL import Image
from os import path
import os

# 1.NETFLIX - Natália

### 1.2 Histórico de pesquisas


campos principais:
- Profile Name	= Nome do perfil
- Displayed Name = Nome do título
- Action	= Ação realizada(dar play ou visualizar as informações do vídeo)


Leitura:

In [ ]:

df_seach_history = pd.read_csv('/content/drive/MyDrive/dados-projeto-final/NETFLIX/CONTENT_INTERACTION/SearchHistory.csv')


Excluindo colunas desnecessárias as análises:

In [ ]:
df_seach_history.drop(columns=['Country Iso Code', 'Device', 'Query Typed'], inplace=True)


Excluindo registros que se refiram a outros perfins presentes:


In [ ]:
df_seach_history.drop(df_seach_history.loc[df_seach_history['Profile Name'] == 'ITALO' ].index, inplace=True)
df_seach_history.drop(df_seach_history.loc[df_seach_history['Profile Name'] == 'KID' ].index, inplace=True)
df_seach_history.drop(df_seach_history.loc[df_seach_history['Profile Name'] == 'Giulliana' ].index, inplace=True)


Analisando os tipos dos campos para posterior conformação necessária:

In [ ]:
df_seach_history.info()

Transformar campo para tipo datetime:


In [ ]:
df_seach_history['Utc Timestamp'] = pd.to_datetime(df_seach_history['Utc Timestamp'], format = '%Y-%m-%d %H:%M:%S')


In [ ]:
df_seach_history

Exportando dataframe em formato csv para uso no tableau:

In [ ]:
df_seach_history.to_csv( '_SeachHistory_netflix.csv' , index = False , header = False )


### 1.3 Atividades de visualização

Leitura do csv:

In [ ]:

df_viewing_activity = pd.read_csv('/content/drive/MyDrive/dados-projeto-final/NETFLIX/CONTENT_INTERACTION/ViewingActivity.csv')


Conferindo campos:

In [ ]:
df_viewing_activity.info()

Tratando tipos incoerentes:


In [ ]:
df_viewing_activity['Bookmark'] = pd.to_datetime(df_viewing_activity['Bookmark'], format = '%H:%M:%S')
df_viewing_activity['Duration'] = pd.to_datetime(df_viewing_activity['Duration'], format= '%H:%M:%S')
df_viewing_activity['Start Time'] = pd.to_datetime(df_viewing_activity['Start Time'], format= '%Y-%m-%d %H:%M:%S')


Eliminando linhas da coluna 'Supplemental Video Type', que continha valores iguais a: RECAP', 'TEASER_TRAILER', 'TRAILER', 'HOOK', 'PROMOTIONAL','TUTORIAL', 'BONUS_VIDEO', 'PREVIEW'. Esses campos, provavelmento, indicam trailer/apresentação de alguns títulos, com duração de visualização por o cliente abaixo de 1 minuto, sendo, dessa forma, desprezível:

In [ ]:
 
df_viewing_activity.drop(df_viewing_activity[df_viewing_activity['Supplemental Video Type'] == 'HOOK'].index, inplace=True)
df_viewing_activity.drop(df_viewing_activity[df_viewing_activity['Supplemental Video Type'] == 'RECAP'].index, inplace=True)
df_viewing_activity.drop(df_viewing_activity[df_viewing_activity['Supplemental Video Type'] == 'TEASER_TRAILER'].index, inplace=True)
df_viewing_activity.drop(df_viewing_activity[df_viewing_activity['Supplemental Video Type'] == 'TRAILER'].index, inplace=True)
df_viewing_activity.drop(df_viewing_activity[df_viewing_activity['Supplemental Video Type'] == 'PROMOTIONAL'].index, inplace=True)
df_viewing_activity.drop(df_viewing_activity[df_viewing_activity['Supplemental Video Type'] == 'PREVIEW'].index, inplace=True)

Deixando apenas as colunas no qual o 'Profile Name' é igual a 'Natália', para que não haja interferência nas análises:


In [ ]:
df_viewing_activity.drop(df_viewing_activity.loc[df_viewing_activity['Profile Name'] == 'ITALO' ].index, inplace=True)
df_viewing_activity.drop(df_viewing_activity.loc[df_viewing_activity['Profile Name'] == 'KID' ].index, inplace=True)
df_viewing_activity.drop(df_viewing_activity.loc[df_viewing_activity['Profile Name'] == 'Giulliana' ].index, inplace=True)

Reorganizando o index após eliminar linhas correspondêntes a outros perfis depreziveis as análises:


In [ ]:
df_viewing_activity = df_viewing_activity.reset_index(drop=True)
df_viewing_activity

Criando função para isolar o título das obras da netflix e adicionando nova coluna com título isolado:

In [ ]:
def titulo_limpo(x):
  tamanho_lista = len(x.split(':'))
  if tamanho_lista <= 3:
    titulo = ':'.join(x.split(':')[:1])
    return titulo 
  elif tamanho_lista <= 4:
    titulo = ':'.join(x.split(':')[:2])
    return titulo
  else: 
    #titulo = x.split(':')[:2]
    titulo = ':'.join(x.split(':')[:2])
    return titulo

# criação de nova coluna 
df_viewing_activity['titulo_limpo_'] = df_viewing_activity['Title'].apply(titulo_limpo)
df_viewing_activity

Elimando colunas desncessárias:

In [ ]:
df_viewing_activity.drop(columns=['Country Iso Code', 'Device', 'Query Typed'], inplace=True)


Criando colunas para hora, minutos e segundos extraídas do tipo datetime:

In [ ]:
df_viewing_activity['minutos'] = df_viewing_activity['Duration'].dt.minute
df_viewing_activity['segundos'] = df_viewing_activity['Duration'].dt.second
df_viewing_activity['horas'] = df_viewing_activity['Duration'].dt.hour

Exportando arquivo tratado:

In [ ]:
df_viewing_activity.to_csv( 'ViewingActiwity_netflix.csv' , index = False , header = False )


# 2. INSTAGRAM - Natália

## 2.1 Suggested accounts viewed 

Leitura do arquivo json:

In [ ]:
path_posts_viewed = '/content/drive/MyDrive/dados-projeto-final/INSTAGRAM /ads_and_content/posts_viewed.json'

In [ ]:
# Dataframe que irá mostrar os posts visualizados.
with open(path_posts_viewed) as f:
  json_dict = json.load(f)
df_posts_viewed = pd.json_normalize(json_dict, "impressions_history_posts_seen")
df_posts_viewed

Excluindo colunas desnecessárias as análises:

In [ ]:

df_posts_viewed.drop(columns=['title',	'string_map_data.Author.href', 'string_map_data.Author.timestamp', 'string_map_data.Time.href',	'string_map_data.Time.value'	], inplace=True)

Criando cópia do dataframe df_posts_viewed, para futuras rotinas não atacarem seu estado:

In [ ]:
df_or_posts_viewed = df_posts_viewed.copy()


Renomeando colunas:

In [ ]:
df_or_posts_viewed.rename(columns={'string_map_data.Time.timestamp': 'data_timestamp'}, inplace = True)


Exportando o arquivo csv: 

In [ ]:
df_or_posts_viewed.to_csv( 'Nat_PostsViewed_instagram.csv' , index = False , header = False )

In [ ]:
openfile=open(r"~\histórico-de-visualização.json", encoding='UTF-8')
df_youtube_views = json.load(openfile)

## 2.2 Vídeos visualizados


Leitura do arquivo json:

In [ ]:
# leitura do dataframe que traz os vídeos que foram visualizados e seu timestamp.
path_videos_watched = '/content/drive/MyDrive/dados-projeto-final/INSTAGRAM /ads_and_content/videos_watched.json'
with open(path_videos_watched) as f_videos:
  json_videos = json.load(f_videos)
df_videos_watched = pd.json_normalize(json_videos, "impressions_history_videos_watched")
df_videos_watched

Fazendo cópia desse dataframe para que futuras mudanças não o afetem:

In [ ]:
df_or_videos_watched = df_videos_watched.copy()


Renomenando colunas:
df_or_videos_watched.rename(columns={'string_map_data.Author.value': 'video_visualizado_ig'}, inplace = True)


Concatenando os dataframes (que sofreram copia) para ser utilizado como fonte de dados sobre contas que visualizo no instagram, já que retratam visualizações, porém, em áreas diferentes:

In [ ]:
conteudo_geral_visto_ig = pd.concat([df_posts_viewed, df_videos_watched])
conteudo_geral_visto_ig 

Renoamentado as colunas do dataframe criado através de uma concatenação:

In [ ]:
conteudo_geral_visto_ig.rename(columns={'string_map_data.Author.value': 'conteudo_visualizado_ig', 'string_map_data.Time.timestamp': 'data_timestamp'}, inplace = True)


Exportando:

In [ ]:
conteudo_geral_visto_ig.to_csv('nat_geral_post_video_Visualizados_ig.csv', index = False , header = True)

# Spotify Felipe

In [4]:
spotify = pd.read_json("https://raw.githubusercontent.com/Felisouza/projeto_final_vamoAI/main/Bases%20de%20dados/Spotify/StreamingHistory_Felipe.json", encoding='UTF-8')
spotify.head()

,endTime,artistName,trackName,msPlayed
0,2020-07-19 00:38,Rosa Neon,Vai Devagar,218775
1,2020-07-19 00:58,Rosa Neon,Brilho de Leão,19224
2,2020-07-19 02:54,Jay Rock,Rotation 112th,212240
3,2020-07-19 02:59,Childish Gambino,II. Earth: The Oldest Computer (The Last Night),282418
4,2020-07-19 03:03,Beastie Boys,Paul Revere,220840


In [ ]:
spotify.to_csv(r'')

## Limpeza de dados

In [ ]:
spotify['endTime'] = spotify['endTime'].astype(np.datetime64)

## Coluna de dias da semana

In [ ]:
dias = {
    'Monday': 'Segunda',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday':'Sábado',
    'Sunday': 'Domingo' 
}

In [ ]:
lista_dias = []
for i in spotify['endTime'].dt.day_name():
    for chave, valor in dias.items():
        if i == chave:
            lista_dias.append(valor)

spotify['dias da semana'] = lista_dias

## Podcasts e músicas

In [ ]:
podcasts = ['Prato Cheio', 'Bem-Estar & Movimento - Marcio Atalla', 'Podpah', 'Radio Terrana', 'Hoje na Luta', 'Medo e Delírio em Brasília', 'Papo Sobremesa', 'Pizza de Dados',  "AfroPython BR", "RefatorandoCast", "AFETOS por Gabi Oliveira e Karina Vieira ", "CAOScast", "QuebraDev", "A Mesa", 	"Negro da Semana", "Fala Preta!", "Sexoterapia", "São Paulo Mil Grau Podcast", "Afrofuturo", "Roda Viva ", 	"Verifica. O podcast da Lupa.", "ANTIVÍRUS", "SpinCast", "90 anos em três cores - A história do São Paulo contada em podcast", "Infiltrados No Cast", 	"Dibradoras", "CAFÉ COM BOULOS", "Podcast Tecnopolítica", 	"Lado Black", "Falecast", "Fogo no Parquim", "DatabaseCast", "Diálogos Pretos", "Quem Somos Nós?", "Urbanidades", "O Lado Negro Da Força", 	"MEMOH", 	"História Preta", "POUCAS", 	"Xadrez Verbal", "Trivela", "Thunder Radio Show", "THShow", "Som das Torcidas", "Rádio Companhia", "Presidente da Semana", "Prancheta do PVC - Paulo Vinícius Coelho", 	"Papricast", 	"Naruhodo", "Meu Time de Botão", 	"Fronteiras Invisíveis do Futebol", "Esquizofrenoias", 	"Escuta", "Politiquês", "Como Começar", "Durma com essa", "AntiCast", "Café da Manhã", "Viracasacas Podcast", "Data Hackers", "ROLÊ ZICA por BOLÍVIA", "QUALÉ, MORÉ?", "Não Ouvo Podcasts", "Pretas na Rede", "Crivelli Convida", "Benzina", "Hipsters Ponto Tech", "Biscoito", "Meu Nome é Correria", "Mais Que 8 Minutos - Podcast", 	"Caixa de Histórias", 	"Headbenzz", "Cinema com Ciência", "Paternidade.doc" ]

In [ ]:
lista = []
for i in spotify['artistName']:
    if i in podcasts:
        lista.append('podcast')
    else:
        lista.append('música')

spotify['tipo'] = lista
spotify.head()

In [ ]:
spotify.info()

In [ ]:
spotify.describe()

In [ ]:
spotify[['msPlayed', 'artistName']].groupby('artistName').sum()

In [ ]:
spotify[['msPlayed', 'trackName']].groupby('trackName').sum()

# Youtube - Felipe

In [ ]:
openfile = open("https://raw.githubusercontent.com/Felisouza/projeto_final_vamoAI/main/Bases%20de%20dados/Youtube/hist%C3%B3rico-de-visualiza%C3%A7%C3%A3o_Felipe.json", encoding='UTF-8')
youtube_felipe_json=json.load(openfile)
youtube_felipe_json

In [ ]:
youtube_felipe.info()

In [ ]:
### Explorando os dados
filtro = youtube_felipe['Nome vídeo'].str.contains('Python')
youtube_felipe[filtro]

python = youtube_felipe[youtube_felipe['Nome vídeo'].str.contains('Python')]
python.query('Data.dt.year == 2021')

## Inscrições Youtube Felipe

In [ ]:
openfile = open(r'C:\Users\feh_s\Desktop\projeto_final\Dados\YouTube e YouTube Music\inscrições\inscrições.json', encoding='UTF-8')
json_inscricoes = json.load(openfile)

In [ ]:
canal = pega_coluna_2chave_dicionarios(json_inscricoes,'snippet', 'title')
descricao = pega_coluna_2chave_dicionarios(json_inscricoes,'snippet', 'description')

inscricoes = pd.DataFrame([canal, descricao]).T
inscricoes.columns = ['Canal', 'Descrição do canal']

inscricoes.head()

In [ ]:
inscricoes.info()

In [ ]:
inscricoes.describe()

In [ ]:
inscricoes.to_csv(
    r'C:\Users\feh_s\Google Drive\7_Resilia\projeto_final_vamoAI\Bases de dados\Youtube\inscrições_Felipe.csv',
    encoding='UTF-8',
    sep=',',
    index=False
    )

## Funções

Função que pega uma lista a partir do json. Quando o json precisa de apenas uma chave. Funciona para um json que tem uma lista de dicionários

In [ ]:

def pega_coluna(json, chave):
        lista = []
        for i in json:
                lista.append(i[chave])
        return lista

Função que pega uma lista a partir do json. Quando o json precisa de 2 chaves. Funciona para um json que tem uma lista de dicionários. Funciona apenas para o histórico de visualizações. O que não é canal (propaganda) ele retorna como adsense:

In [ ]:

def pega_coluna_2chave(json, chave1, chave2):
    lista = []
    for i in json:
        if chave1 in i:
            if len(i[chave1]) > 1: lista.pop()
            for k in i[chave1]:
                lista.append(k[chave2])
        else:
            lista.append('adsense')
    return lista

Função que pega uma lista a partir do json. Quando o json precisa de 2 chaves
Serve para arquivo json em que está estruturado com dicionários de dicionários e não lista de dicionários.

In [ ]:

def pega_coluna_2chave_dicionarios(json, chave1, chave2):
    lista = []
    for i in json:
        for chave, valor in i[chave1].items():
            if chave == chave2:
                lista.append(valor)
    return lista

## DataFrame histórico de visualizações

In [ ]:
nome_video = pega_coluna(df_youtube_views, 'title')
tempo = pega_coluna(df_youtube_views, 'time')
canal = pega_coluna_2chave(df_youtube_views, 'subtitles','name')

youtube_felipe = pd.DataFrame(data = [canal, nome_video, tempo]).T
youtube_felipe.columns=['Canal', 'Nome vídeo', 'Data']
youtube_felipe.head()

In [ ]:
youtube_felipe['Data'] = youtube_felipe['Data'].astype(np.datetime64)

### Limpeza dos dados

In [ ]:
youtube['Data'] = youtube['Data'].astype(np.datetime64)

In [ ]:
youtube.info()

## Inscrições

In [ ]:
canal = pega_coluna_2chave_dicionarios(json_inscricoes,'snippet', 'title')
descricao = pega_coluna_2chave_dicionarios(json_inscricoes,'snippet', 'description')

inscricoes = pd.DataFrame([canal, descricao]).T
inscricoes.columns = ['Canal', 'Descrição do canal']

inscricoes.head()

# Youtube Natália

In [ ]:
openfile = open(r'C:\Users\feh_s\Google Drive\7_Resilia\projeto_final_vamoAI\Bases de dados\Youtube\natalia-histórico-de-visualização.json', encoding='UTF-8')
df_youtube_views = json.load(openfile)

nome_video = pega_coluna(df_youtube_views, 'title')
tempo = pega_coluna(df_youtube_views, 'time')
canal = pega_coluna_2chave(df_youtube_views, 'subtitles','name')

youtube_natalia = pd.DataFrame(data = [canal, nome_video, tempo]).T
youtube_natalia.columns=['Canal', 'Nome vídeo', 'Data']


youtube_natalia['Data'] = youtube_natalia['Data'].astype(np.datetime64)

youtube_natalia.tail(10)

In [ ]:
youtube_natalia = youtube_natalia.drop([6712, 6713,6714,6715,6716,6717])

In [ ]:
youtube_natalia['Data'] = youtube_natalia['Data'].astype(np.datetime64)

# 3. Youtube - Victor

Acessando os arquivos via drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Mounted at /content/drive


Lendo arquivos com as informações do histórico de visualização:

In [ ]:
openfile=open("/content/drive/MyDrive/youtube_dados/TV_histórico-de-visualização.json", encoding='UTF-8')
df_youtube_views1 = json.load(openfile)

In [ ]:
openfile=open("/content/drive/MyDrive/youtube_dados/Celular_histórico-de-visualização.json", encoding='UTF-8')
df_youtube_views2 = json.load(openfile)

In [ ]:
openfile=open("/content/drive/MyDrive/youtube_dados/PC_histórico-de-visualização.json", encoding='UTF-8')
df_youtube_views3 = json.load(openfile)

In [ ]:
nome_video = pega_coluna(df_youtube_views1, 'title')
tempo = pega_coluna(df_youtube_views1, 'time')
canal = pega_coluna_2chave(df_youtube_views1, 'subtitles','name')

youtube_tv = pd.DataFrame(data = [canal, nome_video, tempo]).T
youtube_tv.columns=['Canal', 'Nome vídeo', 'Data']
youtube_tv.head()

In [ ]:
nome_video = pega_coluna(df_youtube_views2, 'title')
tempo = pega_coluna(df_youtube_views2, 'time')
canal = pega_coluna_2chave(df_youtube_views2, 'subtitles','name')

youtube_celular = pd.DataFrame(data = [canal, nome_video, tempo]).T
youtube_celular.columns=['Canal', 'Nome vídeo', 'Data']
youtube_celular.head()

In [ ]:
nome_video = pega_coluna(df_youtube_views3, 'title')
tempo = pega_coluna(df_youtube_views3, 'time')
canal = pega_coluna_2chave(df_youtube_views3, 'subtitles','name')

youtube_pc = pd.DataFrame(data = [canal, nome_video, tempo]).T
youtube_pc.columns=['Canal', 'Nome vídeo', 'Data']
youtube_pc.head()

Unindo fontes de visualizações do youtube, com objetivo de gerar um dataframe único:

In [ ]:
victor_youtube = pd.concat([youtube_celular, youtube_pc])
victor_youtube.head(4)

Eliminando palavra 'Watched' que se apresenta em todos os títulos do histórico, já que não pertence aos mesmos:

In [ ]:
victor_youtube['Nome vídeo'] = victor_youtube['Nome vídeo'].str.replace('Watched', '')
victor_youtube

In [ ]:
Exportando arquivo:

In [ ]:
victor_youtube.to_csv('victor_youtube1.csv', index = False, header=True)

# Youtube todos os integrantes

In [ ]:
youtube = pd.concat([youtube_felipe, youtube_natalia, victor_youtube])

In [ ]:
youtube.info()

In [ ]:
#Cria coluna de dados e python
procura_python = ['Python', 'python']
procura_dados = ['Dados', 'dados', 'data', 'Data']
youtube['python'] = np.where(youtube['Nome vídeo'].str.contains('|'.join(procura_python)),1,0)
youtube['dados'] = np.where(youtube['Nome vídeo'].str.contains('|'.join(procura_dados)),1,0)

In [ ]:
youtube.head()

### Dias da semana

In [ ]:
dias = {
    'Monday': 'Segunda',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday':'Sábado',
    'Sunday': 'Domingo' 
}

In [ ]:
lista_dias = []
for i in youtube['Data'].dt.day_name():
    for chave, valor in dias.items():
        if i == chave:
            lista_dias.append(valor)

lista_dias
youtube['dias da semana'] = lista_dias
youtube.head(30)

### meses

In [ ]:
meses = {
    'January': 'Janeiro',
    'February': 'Fevereiro', 
    'March': 'Março',
    'April': 'Abril',
    'May': 'Maio',
    'June': 'Junho',
    'July': 'Julho',
    'August': 'Agosto',
    'September': 'Setembro',
    'October': 'Outubro',
    'November': 'Novembro',
    'December': 'Dezembro'
}

In [ ]:
lista_meses = []
for i in youtube['Data'].dt.month_name():
    for chave, valor in meses.items():
        if i == chave:
            lista_meses.append(valor)

lista_meses
youtube['meses'] = lista_meses
youtube.head()

In [ ]:
mascara = youtube['Nome vídeo'].str.contains('Monty')
youtube[mascara]
youtube.loc[[10740, 10741, 10992, 10993, 10994, 10996, 10997, 10998, 10999, 11000, 11683], 'python'] = 0

# Nuvem de palavras

In [ ]:
palavras1 = ['watched', 'watch', 'da', 'das', 'de', 'di' 'do', 'dos', 'du', 'youtube', 'a', 'e' 'i', 'o', 'os', 'u', 'um', 'uma', 'https', 'que', 'v', 'e', 'é', 'como', 'video', 'vídeo', 'para', 'em', 'não', 'por', 'parte', 'removed', 'na', 'no', 'mais', 'se', 'dia', 'fazer', 'ou', 'pokémon', 'pokemon', 'go', 'minecraft', 'criança', 'dinossauro', 'jogo', 'clash royale', 'jurassic', 'roblox', 'sonic', 'seu', 'sua', 'dinossauros', 'crianças']

## Nuvem antes do curso

In [ ]:
youtube_nuvem = youtube[youtube['Data'].dt.year != 2021]
youtube_nuvem['Data'].dt.year.unique()


text = ' '.join(youtube_nuvem['Nome vídeo'].str.lower())

stopwords = set(STOPWORDS)
stopwords.update(palavras1)

d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()
mask = np.array(Image.open(path.join(d, "youtube logo_icon.png")))
wordcloud2 = WordCloud(stopwords=stopwords, background_color='#191414', colormap='Reds', mask=mask).generate(text)
plt.figure(figsize=(15,10))

plt.imshow(wordcloud2, interpolation='bilinear')
plt.axis("off")
plt.show()
plt.savefig(fname= 'grafico', transparent=True)

## Nuvem 2021

In [ ]:
#juntando os 3 bancos filtrados por ano
youtube2021  = youtube[youtube['Data'].dt.year == 2021]

#passando a coluna, em minusculo, para uma variável. A variável recebe um grande texto com todas as descrições dos vídeos
text = ' '.join(youtube2021['Nome vídeo'].str.lower())

#Setando e inserindo novas palavras para ficar de fora da nuvem.
stopwords = set(STOPWORDS)
stopwords.update(palavras)

#construindo a máscara para plotar o gráfico
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd() #passando o caminho para uma variável. A imagem deve estar no mesmo local que o notebook
mask = np.array(Image.open(path.join(d, "youtube logo_icon.png"))) #abrindo e passando a imagem para a máscara da nuvem

#Passando os atributos para plotar a nuvem de palavras
wordcloud2 = WordCloud(stopwords=stopwords, background_color='#191414', colormap='Reds', mask=mask).generate(text)

#Plotando o gráfico
plt.figure(figsize=(15,10))
plt.imshow(wordcloud2, interpolation='bilinear')
plt.axis("off")
plt.show()